In [1]:
!pip install git+https://github.com/ibm-granite-community/utils "langchain_community<0.3.0" replicate colabcode

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-b1p8c_1i
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-b1p8c_1i
  Resolved https://github.com/ibm-granite-community/utils to commit 1514191fbbc4605ed4fdfdcb448f2ee41477058f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
os.environ["NGROK_AUTH_TOKEN"] = "2zLwnLDph938yfIobq2o8Ue8aZS_2GPedgYx8NDYQz3KHv4sV"

from pyngrok import conf, ngrok

conf.get_default().auth_token = os.environ["NGROK_AUTH_TOKEN"]

In [3]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":1024, "temperature":0.2},
)

REPLICATE_API_TOKEN loaded from Google Colab secret.


In [4]:
def zeroshot_prompt_diet(data):
    return f"""
    Saya pengguna aplikasi diet bernama Kaloriku. Hari ini saya telah mengonsumsi {data['kalori_masuk']}kkal dan membakar {data['kalori_terbakar']}kkal melalui aktivitas.
    Target saya adalah defisit {data['target_defisit']}kkal.
    Tolong berikan saran makanan yang harus saya kurangi atau aktivitas tambahan yang bisa saya lakukan agar target defisit tercapai.
    Berikan jawaban ringkas dan mudah dipahami.
    """

# Step 5: Setup Flask API
from flask import Flask, request, jsonify
import threading

app = Flask(__name__)

@app.route('/saran-ai', methods=['POST'])
def saran_ai():
    data = request.json
    prompt = zeroshot_prompt_diet(data)
    result = model.invoke(prompt)
    return jsonify({'saran': result})

def run_flask():
    app.run(host='0.0.0.0', port=5000)

# Jalankan Flask di background
thread = threading.Thread(target=run_flask)
thread.start()

# Step 6: Expose Colab Server via ngrok
from colabcode import ColabCode
ColabCode(port=5000, password="kaloriku123")


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


ERROR:pyngrok.process.ngrok:t=2025-07-03T06:35:12+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-03T06:35:12+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-03T06:35:12+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

Saran AI Granite:
Selamat menggunakan aplikasi diet harian! Berdasarkan informasi yang diberikan, Anda mencapai defisit kalori target Anda (500 kkal) dengan makanan 1800 kkal dan olahraga 300 kkal, yang menghasilkan defisit sebanyak 1000 kkal (1800 - 300 = 1500, dan 1500 - 500 = 1000).

Saran untuk makanan dan aktivitas:

1. **Makanan**: Tetap konsumsi makanan Anda dengan total kalori 1800, tetapi pastikan makanan Anda terdistribusi seimbang antara macam-macam macronutrient (karbohidrat, protein, dan lemak). Coba untuk mencakup:

   - Sumber karbohidrat sehat seperti buah, sayuran, dan grain whole.
   - Sumber protein seperti daging, ikan, tofu, atau tempe.
   - Makanan yang kaya dengan fiber untuk membantu rasa satisfied dan keseimbangan hormon leptin dan ghrelin.

2. **Aktivitas**: Untuk mendukung tujuan kebugaran Anda, pertimbangkan untuk menambahkan aktivitas fisik lebih intensif secara berkala. Misalnya:

   - Lakukan olahraga cardio seperti joging, bersiul, atau berolak-olah sela